<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/biagiominio/Remote_sensing_2021/blob/main/R_code_multivariate_analysis.ipynb#scrollTo=WfzIAI-J6TEO">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biagiominio/Remote_sensing_2021/blob/main/R_code_multivariate_analysis.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Dimensionality Reduction 

In multispectral or multidimensional remote sensing data, a different degree of correlation between the bands is often observed. This may depend on real factors, such as the physical nature of the observed surfaces, the width of the spectral band, the noise inherent in the data.
The problem can be overcome by choosing the spectral bands that are statistically least correlated with each other. However, it is not easy to identify which variables are less significant in expressing the overall variability of the data.
It is preferable to use a transformation procedure that allows you to summarize the information of the original data with a smaller number of dimensions. If the new dimensions define a new set of variables, not related to each other,
these are called principal components or PCs.
Each PCs is constructed as a linear combination of the original variables (spectral bands) and the procedure used is named as [Principal components analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA).

![](https://i.pinimg.com/originals/c0/0a/47/c00a47aa8cf775c124473a060d26a3b9.jpg)



### Install R packages
The [install.packages](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/install.packages) function is used to download and install packages from CRAN-like repositories.
To install multiple packages at the same time, you define a vector that contains the names of the packages to be installed.

The [raster](https://www.rdocumentation.org/packages/raster/versions/3.4-10) package defines classes and methods for spatial raster data access and manipulation. 
[RStoolbox](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6) is an package providing a wide range of tools for your every-day remote sensing processing needs. The available tool-set covers many aspects for remote sensing image processing and analysis such as calculating spectral indices, principal component transformation, unsupervised and supervised classification or fractional cover analyses.

In [ ]:
packages <- c("raster", "RStoolbox")
install.packages(packages)

The [library](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/library) function is used to load previously installed packages.
Using the [lapply](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/lapply) function we simultaneously apply the `library()` function to all elements of the vector containing the packages.



In [ ]:
lapply(packages , library, character.only = TRUE)

Set the current working directory of the R process with the [setwd](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/getwd) function.

In [ ]:
setwd("/content")

## Study area
Let's analyze Landsat images of the [*Parakanã Indigenous Reserve*](https://pib.socioambiental.org/en/Povo:Parakan%C3%A3) in the state of Pará, a northern region of Brazil. This reserve, founded in 1975, is listed as an indigenous area but has not been assigned to an International Union for Conservation of Nature (IUCN) category (Protected Planet, http://www.protectedplanet.net). 

Large-scale deforestation in the region is due to agricultural expansion and cattle ranching and has modified this region considerably in the past. Mainly, the Parakanã Indigenous Reserve remains forested. A reservoir on the Rio Tocantins (Tocantins River) covers the north-east part of the area. This large body of water (Lago Tucuruí) is an artificial lake created by the construction of a hydroelectric dam called Represa de Tucuruí (Tucuruí Dam) which was finished in 1984. Construction of the dam is closely related to the observed deforestation and degradation patterns, as workers began settling down after the construction of the dam was complete. Moreover the area is bisected by the Transamazônica highway. 

Download Landsat images of the Parakanã Indigenous Reserve from this [link]() to get the data to use in this example.

<img src="https://media.gettyimages.com/photos/large-deforestation-surrounds-the-parakana-indigenous-reserve-in-the-picture-id959485258?k=6&m=959485258&s=612x612&w=0&h=MehPfqdZHxUZikRg4-j3qTsXPj5h6f4sHcUrE31h82U=" width="1000"/>

<font size="2"> *AMAZON, BRAZIL- JULY 29, 2017: Deforestation surrounding the Parakana Indigenous Reserve in the Brazilian State of Para, Northern Region of Brazil. (Photo by Gallo Images/USGS/NASA Landsat data/Orbital Horizon)*<font>



## Reading spatial data
To read and view a multilevel object it is common to use the [brick](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/brick) function.
Select the images already pre-processed for the year 2011. Explore the properties of the `p224r63_2011` object.

Also there may be multiple functions with the same name in multiple packages. The colon operator allows you to specify the specific function you want: `package::function_name`



In [ ]:
p224r63_2011 <- raster::brick("p224r63_2011_masked.grd")
p224r63_2011

LANDSAT BANDS: \\
$\mathsf{B1}$ = Blue; 
$\mathsf{B2}$ = Green;
$\mathsf{B3}$ = Red;
$\mathsf{B4}$ = Near Infrared;
$\mathsf{B5}$ = Short Wave Infrared;
$\mathsf{B6}$ = Thermal Infrared;
$\mathsf{B7}$ = Short Wave Infrared.

## Multicollinearity
The spectral bands of an image have some degree of correlation with each other.
Excessive correlation between two or more bands results in a
greater redundancy of overall information.
This feature, called [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity) or collinearity, can be useful for eliminating unnecessary information to view images. In this way you get a good compromise between size and quality of an image.

It is possible to visualize the degree of correlation between each pair of bands through different scatter plots. We can either manually build the individual graphs like this:
```
plot(p224r63_2011$B1_sre, p224r63_2011$B2_sre, col="red", pch=19, cex=2)
       .                            .                          .
       .                            .                          .
       .                            .                          .
plot(p224r63_2011$B6_sre, p224r63_2011$B5_sre, col="red", pch=19, cex=2)
```
Otherwise, we can use the [pairs](https://www.rdocumentation.org/packages/graphics/versions/3.6.2/topics/pairs) function to automatically produce a scatterplot matrix with the respective correlation coefficients.


In [ ]:
pairs(p224r63_2011)

## Principal Component Analysis
As mentioned earlier, PCA transforms an originally related data set into a substantially smaller set of unrelated variables. This procedure reduces the dimensionality of the data (i.e. reduces the number of bands) by increasing the signal-to-noise ratio. 

Apply the PCA to the Raster* object via the [rasterPCA](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6/topics/rasterPCA) function of the `RStoolbox` package.

In [ ]:
p224r63_2011.pca <- RStoolbox::rasterPCA(p224r63_2011)
p224r63_2011.pca

Explore the parameters of the PCA model by running the `summary()` of the `p224r63_2011.pca` object.

In [ ]:
summary(p224r63_2011.pca$model)

Through this transformation the measurements contained in the spectral bands are redistributed in the new components (synthetic bands) so that most of the information of the multispectral image is contained in the first three principal components (variance explained = 99.9956%).



### Loadings matrix

The load matrix provides information on how much each band "weighs" and how each band is associated with each of the PCs.
For example, the PC1 is described mainly by the thermal infrared band (B6_bt = 0.999).

In [ ]:
loadings(p224r63_2011.pca$model)

### Visualization of spatial data
Several generic functions have been implemented for "raster" objects to create maps and other plot types. For example, use [plot](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plot) function to create a map of a raster object.
In addition, individual bands can be combined into a single image by assigning individual layers or bands to one of the three color channels (red, green and blue). 
The [plotRGB](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plotRGB) function can be used to make "true (or false) color images" from multi-band satellite images.
In this case, the resulting color tone in the image will be related to the combination of the first three PCs assigned to the three RGB bands.

Export the image using the [pdf](https://www.rdocumentation.org/packages/grDevices/versions/3.6.2/topics/pdf) function to enlarge it better and view the results.

N.B. always use the `dev.off()` function to manage the correct closing and loading of the graphics device.


In [ ]:
pdf(file = '/content/ImagePCA_res.pdf')
plotRGB(p224r63_2011.pca$map, r = 1, g = 2, b = 3, stretch="lin")
dev.off()

In practice, the PCA technique highlights elements or details not visible in the original image. It also solves color display problems of multispectral images in which only three color dimensions (blue, green, and red) are available.

Use the [par](https://www.rdocumentation.org/packages/graphics/versions/3.6.2/topics/par) function to compare the original true and false color images with the image resulting from the combination of three PCs (PC1, PC2 and PC3).


In [ ]:
pdf(file = '/content/Comp_ImagePCA_res.pdf')
par(mfrow = c(3,1))
plotRGB(p224r63_2011, r = 3, g = 2, b = 1, stretch = "lin") # true color images
plotRGB(p224r63_2011, r = 4, g = 3, b = 2, stretch = "lin") # false color images
plotRGB(p224r63_2011.pca$map, r = 1, g = 2, b = 3, stretch = "lin") # latent color images
dev.off()

### Image resampling
If you want to perform the entire procedure on your machine and not on Colab's, you can apply image scaling techniques in order to reduce the calculation time of your statistics.

Image scaling can be interpreted as a form of image resampling or image reconstruction.
*Resampling* is the mathematical technique used to create a new version of the image with a different width and/or height in pixels. Increasing the size of an image is called *upsampling*; reducing its size is called *downsampling*.

The image size reduction occurs through the [aggregate](https://www.rdocumentation.org/packages/raster/versions/3.4-10/topics/aggregate) function of the `raster` package which groups the initial pixels in order to create larger cells.
In the function, the `fact` argument is the aggregation factor expressed as the number of cells in each direction (e.g. the new cell will be equal to 10 X 10 pixels).



In [ ]:
p224r63_2011.res <- aggregate(p224r63_2011, fun=mean, fact=10)
p224r63_2011.res

The spatial resolution changes from 30 $\mathsf{m}$ to 300 $\mathsf{m}$ while the number of pixels is reduced by two orders of magnitude (in this case from 4447533 to 44550).

In [ ]:
# PCA resampling
p224r63_2011.res.pca <- RStoolbox::rasterPCA(p224r63_2011.res)

Use the `par()` function to compare the different spatial resolution between the initial image and the resampled one.

Export the image using the `pdf()` function to enlarge it better and view the differences.


In [ ]:
pdf(file = '/content/downsampling.pdf')
par(mfrow=c(2,2))
plotRGB(p224r63_2011, r=3, g=2, b=1, stretch="lin")
plotRGB(p224r63_2011.res, r=1, g=2, b=3, stretch="lin")
plotRGB(p224r63_2011.pca$map, r=1, g=2, b=3, stretch="lin")
plotRGB(p224r63_2011.res.pca$map, r=1, g=2, b=3, stretch="lin")
dev.off()